# 유틸리티 행렬

In [1]:
import numpy as np
import pandas as pd
import copy
import collections
from scipy import linalg
import math
from collections import defaultdict

In [15]:
# pandas 의 DataFrame 으로 변환
# 처리후 CSV 파일로 저장
#data
df = pd.read_csv('./machine_learning_for_the_web-master/chapter_5/data/ml-100k/u.data',sep='\t',header=None)

# ; 구분 영화명 ID 로 구성
#movie list
df_info = pd.read_csv('./machine_learning_for_the_web-master/chapter_5/data/ml-100k/u.item', sep='|',header=None, encoding='iso-8859-1')
movielist = [df_info[1].tolist()[indx]+';'+str(indx+1) for indx in range(len(df_info[1].tolist()))]
nmovies = len(movielist)
nusers = len(df[0].drop_duplicates().tolist())  

# 값이 0 은 결측치 (평가하지 않음)
# 평점 50회 이하로 받은 영화는 행렬에서 제거 (일부에선 결측치를 초기화 하기도함)
min_ratings = 50
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
dfout = pd.DataFrame(columns=['user']+movielist)

toremovelist = []
for i in range(1,nusers):
    tmpmovielist = [0 for j in range(nmovies)]
    dftmp =df[df[0]==i]
    for k in dftmp.index:
        if counts[dftmp.loc[k][1]]>= min_ratings:           
           tmpmovielist[dftmp.loc[k][1]-1] = dftmp.loc[k][2]
           
        else:
           toremovelist.append(dftmp.loc[k][1])
            
    dfout.loc[i] = [i]+tmpmovielist
  
toremovelist = list(set(toremovelist))
dfout.drop(dfout.columns[toremovelist], axis=1, inplace=True)
dfout.to_csv('./machine_learning_for_the_web-master/chapter_5/data/utilitymatrix.csv',index=None)

In [18]:
df = pd.read_csv('./machine_learning_for_the_web-master/chapter_5/data/utilitymatrix.csv',encoding='iso-8859-1')
df.head(2)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,Richard III (1995);10,...,Cool Runnings (1993);1035,Hamlet (1996);1039,Forget Paris (1995);1041,Multiplicity (1996);1047,She's the One (1996);1048,Koyaanisqatsi (1983);1065,Shallow Grave (1994);1073,Reality Bites (1994);1074,Six Degrees of Separation (1993);1101,Some Kind of Wonderful (1987);1119
0,1,5,3,4,3,3,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,2,4,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# 사용자별 평균 평점 과 아이템별 평균 평점으로 imputation(대체)
# 행렬 R N 사용자 M 아이템 
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in xrange(len(Ri)):
            Ri[i][Ri[i]==0] = sum(Ri[i])/float(len(Ri[i][Ri[i]>0]))
        return Ri
    def itemav():
        for i in xrange(len(Ri[0])):
            Ri[:,i][Ri[:,i]==0] = sum(Ri[:,i])/float(len(Ri[:,i][Ri[:,i]>0]))
        return Ri            
    switch = {'useraverage':userav(),'itemaverage':itemav()}
    return switch[inp]

# 유사도 척도
사용자, 아이템
쿠사인유사도<br>
피어슨상관관계 : 두 벡터의 평균 두 척도는 평균이 0인 경우 일치<br>

In [20]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 

# sim() 두 벡터의 유사도 평가
def sim(x,y,metric='cos'):
    if metric == 'cos':
        # scipy 의  cosine 정의는 앞의 정의와 반대 (값을 -1에서빼야함)
        return 1.-cosine(x,y)
    else:#correlation
        return pearsonr(x,y)[0]

# 협업필터링 방법

## 사용자 기반 협업 필터링
사용자 평점 벡터로 함수

In [23]:
def CF_userbased(u_vec,K,data,indxs=False):
    
    #  FindKNeighbours 에서 이 벡터와 가장 비슷한 사용자 k 를 찾음 
    def FindKNeighbours(r,data,K):
        neighs = []
        cnt=0
        for u in xrange(len(data)):
            if data[u,r]>0 and cnt<K:
                neighs.append(data[u])   
                cnt +=1 
            elif cnt==K:
                break
        return np.array(neighs)
        
        
    # 위 식을 이용, 예측 평점 계산    
    def CalcRating(u_vec,r,neighs):
        rating = 0.
        den = 0.
        for j in xrange(len(neighs)):
            rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den += abs(neighs[j][-1])
        
        # 단, 유틸리티 행렬이 희소해 이웃이 없는 겨우 사용자 평균평점이용
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:
            rating = np.round(u_vec[u_vec>0].mean(),0)
            
        # 예측 평점이 5를 넘거나 1보다 낮은 경우 각각 5와 1로 조정
        if rating>5:
            return 5.
        elif rating<1:
            return 1.
        return rating 
    
    #add similarity col
    data = data.astype(float)
    nrows = len(data)
    ncols = len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data
    
    #calc similarities:
    for u in xrange(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec)==False: #list(data_sim[u,:-1]) != list(u_vec):
           data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
           data_sim[u,ncols] = 0.
        
    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    
    for r in xrange(ncols):
        if u_vec[r]==0:
            #FindKNeighbours: 
            neighs = FindKNeighbours(r,data_sim,K)
            #CalcRating: Calculate the predicted rating 
            u_rec[r] = CalcRating(u_vec,r,neighs)
            
    if indxs:
        #take out the rated movies
        seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
        u_rec[seenindxs] = -1
        recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
        return recsvec    
    return u_rec

## 아이템 기반 협업 필터링

In [24]:
class CF_itembased(object):
    
    # 아이템 유사도 행렬 simatrix 계산
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in xrange(nitems):
            for j in xrange(nitems):
                if j>=i:#triangular matrix
                   self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else:
                   self.simmatrix[i,j] = self.simmatrix[j,i]

    
    # GetKSimItemsperUser: 사용자가 평가하지 않은 아이템과 유사도가 가장 높은 아이템 중 
    # 사용자가 과거에 평가했던 K개의 아이템을 찾는다
    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            # u_vec 사용자 평점 벡터 유틸리티 행렬의 행벡터
            if u_vec[i]>0 and cnt<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K:
                break
        return neighitems
        
        
    # CalcRating: 사용자 평점이 없는 경우 예측하며 가중 평균 평점을 계산하여 
    # 이웃을 찾을 수 없는 경우 이것으로 설정.
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
               neighitems = self.GetKSimItemsperUser(r,K,u_vec)
               #calc predicted rating
               u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec
        return u_rec

## 단순한 아이템 기반 협업 필터링: 슬롭원

In [25]:
class SlopeOne(object):
    # difmatrix: 아이템 i,j의 차 $d_{ij}$를 계산하는데 사용
    
    def __init__(self,Umatrix):
        #calc item similarities matrix
        nitems = len(Umatrix[0])
        self.difmatrix = np.zeros((nitems,nitems))
        self.nratings = np.zeros((nitems,nitems))
    
        def diffav_n(x,y):
            xy = np.vstack((x, y)).T
            xy = xy[(xy[:,0]>0) & (xy[:,1]>0)]
            nxy = len(xy)
            if nxy == 0:
                #print 'no common'
                return [1000.,0]
            return [float(sum(xy[:,0])-sum(xy[:,1]))/nxy,nxy]
            
        for i in xrange(nitems):
            for j in xrange(nitems):
                if j>=i:#triangular matrix                 
                    self.difmatrix[i,j],self.nratings[i,j] = diffav_n(Umatrix[:,i],Umatrix[:,j])
                else:
                    self.difmatrix[i,j] = -self.difmatrix[j,i]
                    self.nratings[i,j] = self.nratings[j,i]
        
    # K개의 최근접 이웃을 찾기 위해 행렬 difmatrix에서 가장 작은 값을 찾음
    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.difmatrix[r])
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K:
                break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            if abs(self.difmatrix[r,i])!=1000: # 평점이 없는 아이템 기본값
                rating +=  (self.difmatrix[r,i]+u_vec[i])*self.nratings[r,i]
                den += self.nratings[r,i]
        if den==0:
            #print 'no similar diff'
            return 0.
        rating = np.round(rating/den,0)
        
        # 예측 평균이 5보다 크거나 1보다 작으면 각각 5,1로 설정
        if rating >5:
            return 5.
        elif rating <1.:
            return 1.
        return rating
        
    def CalcRatings(self,u_vec,K):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                #calc predicted rating
                u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        return u_rec